In [1]:
import os
import json
from insights_prompt_library import *
import inspect
import boto3

In [2]:
bedrock_agent_runtime_client = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name= os.environ.get("Region")
)


In [3]:
with open("prompts_config.json") as file:
    config = json.load(file)

In [23]:
value_to_find = {'insurance_type' : 'Commercial', 'insurance_provider': 'Humana'}
print(value_to_find)

{'insurance_type': 'Commercial', 'insurance_provider': 'Humana'}


In [32]:
def get_prompts(value_to_find):
    retrieved_prompts_list = {}
    input_keys = set(value_to_find.keys())
    matching_keys = [key for key, value in config.items() if input_keys.issubset(set(value['params']))]
    print(matching_keys)
    for key in matching_keys:
        raw_prompt = globals()[key]
        print(raw_prompt)
        #Use a subset of value_to_find containing only relevant keys
        relevant_values = {k: v for k, v in value_to_find.items()  if k in config[key]["params"]}
        filled_prompt = raw_prompt.format(**relevant_values)
        retrieved_prompts_list[key] = filled_prompt
    return retrieved_prompts_list

In [31]:
retrieved_prompts_list_var=get_prompts(value_to_find)
print(retrieved_prompts_list_var.items())

['insurance_type_and_provider_prompt_generator', 'insurance_type_provider_pharmacy_ttf_prompt_generator', 'insurance_type_provider_pharmacy_prompt_generator']
Human:

    You are Agent assist tracking the Patient and agent conversation and help the agent by generating an insight based on the patient's data for insurance type "{insurance_type}" and insurance provider
    "{insurance_provider}" 
    Provide all the details including details around PA

    Note: Only consider the cases when the patient is with Hub.
    
    Assistant:
    
Human:

    You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.
    For a hub patient with insurance_type as  "{insurance_type}" and insurance provider as "{insurance_provider}" 
    Recommend the specialty pharmacies with theirTime to Fill days and compare the time to fill days with the patient's specialty pharmacy "{pharmacy_name}"
    
    Note: Only consider the cases when the patient is 

KeyError: 'pharmacy_name'

In [22]:
def generate_insight_fromkb(prompt):
    response_withkb = bedrock_agent_runtime_client.retrieve_and_generate(
      input={
        'text': prompt
      },
      retrieveAndGenerateConfiguration={
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': 'TZYCKWDKBC',
            'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0'
        },
        'type': 'KNOWLEDGE_BASE'
      }
    )
    return response_withkb["output"]["text"]
    
# insights_withkb = response_withkb["output"]["text"]
# print("Generated Insights", {"generated_insights_withkb": insights_withkb})


In [23]:
def orchestrator(value_to_find):
    retrieved_prompts_list = get_prompts(value_to_find)
    insights_response = {}
    for prompt_type, prompt in retrieved_prompts_list.items():
        insights_response[prompt_type] = generate_insight_fromkb(prompt)
    return insights_response

In [24]:
insights_response = orchestrator(value_to_find)

In [26]:
for i,j in insights_response.items():
    print(f"{i} : {j}")

insurance_type_provider_pharmacy_ttf_prompt_generator : For a hub patient with insurance type "Commercial" and insurance provider "Humana", the specialty pharmacies and their corresponding Time to Fill days are:

- Walgreens: 12 days
- CVS: 15 days
- Elixir: 11 days
- Cigna: 17 days

Compared to the patient's current specialty pharmacy CVS (15 days), Elixir has a lower Time to Fill of 11 days, and Walgreens has a lower Time to Fill of 12 days.
insurance_type_provider_pharmacy_prompt_generator : For patients with "Commercial" insurance type from "Humana" and using the specialty pharmacy "CVS" through the Hub model, the following metrics are available:

Conversion %: 76
Time To Fill: 10
% of patients staying for more than a year: 69
% of Patient Requires PA: 66
TTF when PA required: 12
PA% Approved: 90
% of Patient on treatment if used with financial Assistance program: 58


In [ ]:
# retrieved_prompts_list = get_prompts(value_to_find)

# print(retrieved_prompts_list.items())

# for prompt_type, prompt in retrieved_prompts_list.items():
#     print(f"{prompt_type} : {prompt}")
#     response = retrieve_and(prompt)

# for prompt_type, prompt in retrieved_prompts_list.items():
#     print(prompt)


In [12]:
# for key in matching_keys:
#     function_to_call = getattr(insights_prompt_library, key)
#     function_definition = inspect.getsource(function_to_call)
#     print(function_definition)

def insurance_type_provider_pharmacy_ttf_prompt_generator(insurance_type,insurance_provider,pharmacy_name):
    prompt_claude3_sonnet = """Human:

    You are Agent assist helping the agent to recommend a better specialty pharmacy with the lower time to fill days.
    For a hub patient with insurance_type as  \"""" + insurance_type + """\" and insurance provider as \"""" + insurance_provider + """\" 
    Recommend the specialty pharmacies with theirTime to Fill days and compare the time to fill days with the patient's specialty pharmacy \"""" + pharmacy_name + """\"
    
    Note: Only consider the cases when the patient is with Hub.

    Provide the response in a structured and readable format.
    
    Assistant:
    """

    return prompt_claude3_sonnet

def insurance_type_provider_pharmacy_prompt_generator(insurance_type,insurance_provider,pharmacy_name):
    prompt_claude3_sonnet = """Human:

    You are Agent assist helping the agent and help the agent by generating an insight ba